<a href="https://colab.research.google.com/github/SergChupin/NLP/blob/lesson_05/nlp_hw_05.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##Тема «POS-tagger и NER»

####Задание 1. Написать теггер на данных с русским языком
- проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
- написать свой теггер как на занятии, попробовать разные векторайзеры, добавить знание не только букв но и слов
-сравнить все реализованные методы, сделать выводы


In [1]:
!pip install pyconll

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import nltk
from nltk.tag import DefaultTagger
from nltk.tag import UnigramTagger
from nltk.tag import BigramTagger, TrigramTagger
from nltk.tag import RegexpTagger

import pyconll

In [3]:
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
!wget https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu

--2022-09-02 17:52:49--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-train-a.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 40736565 (39M) [text/plain]
Saving to: ‘ru_syntagrus-ud-train-a.conllu.4’

ru_syntagrus-ud-tra 100%[===================>]  38.85M   114MB/s    in 0.3s    

2022-09-02 17:52:50 (114 MB/s) - ‘ru_syntagrus-ud-train-a.conllu.4’ saved [40736565/40736565]

--2022-09-02 17:52:50--  https://raw.githubusercontent.com/UniversalDependencies/UD_Russian-SynTagRus/master/ru_syntagrus-ud-dev.conllu
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185

In [4]:
full_train = pyconll.load_from_file('/content/ru_syntagrus-ud-train-a.conllu')
full_test = pyconll.load_from_file('/content/ru_syntagrus-ud-dev.conllu')

In [5]:
fdata_train = []
for sent in full_train[:]:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test[:]:
    fdata_test.append([(token.form, token.upos) for token in sent])
    
fdata_sent_test = []
for sent in full_test[:]:
    for word in sent:
        fdata_sent_test.append(word.form)

In [6]:
MAX_SENT_LEN = max(len(sent) for sent in full_train)
MAX_ORIG_TOKEN_LEN = max(len(token.form) for sent in full_train for token in sent)
print('Максимальная длина предложения', MAX_SENT_LEN)
print('Максимальная длина токена', MAX_ORIG_TOKEN_LEN)

Максимальная длина предложения 194
Максимальная длина токена 31


In [7]:
unigram_tagger = UnigramTagger(fdata_train)
unigram_tagger.evaluate(fdata_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


0.823732013802982

In [8]:
bigram_tagger = BigramTagger(fdata_train, backoff=unigram_tagger)
bigram_tagger.evaluate(fdata_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


0.8292792499511688

In [9]:
trigram_tagger = TrigramTagger(fdata_train, backoff=unigram_tagger)
trigram_tagger.evaluate(fdata_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


0.828550035809623

In [10]:
trigram_tagger = TrigramTagger(fdata_train, backoff=bigram_tagger)
trigram_tagger.evaluate(fdata_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  


0.8291425222996289

In [11]:
def backoff_tagger(train_sents, tagger_classes, backoff=None):
    for cls in tagger_classes:
        backoff = cls(train_sents, backoff=backoff)
    return backoff


backoff = DefaultTagger('NN') 
tag = backoff_tagger(fdata_train,  
                     [UnigramTagger, BigramTagger, TrigramTagger],  
                     backoff = backoff) 
  
tag.evaluate(fdata_test)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:12: DeprecationWarning: 
  Function evaluate() has been deprecated.  Use accuracy(gold)
  instead.
  if sys.path[0] == '':


0.827905462595221

In [12]:
train_token = []
train_label = []
for sent in fdata_train[:]:
    for token in sent:
        train_token.append(token[0])
        train_label.append('NO_TAG' if token[1] is None else token[1])
        
test_token = []
test_label = []
for sent in fdata_test[:]:
    for token in sent:
        test_token.append(token[0])
        test_label.append('NO_TAG' if token[1] is None else token[1])

In [13]:
from sklearn.feature_extraction.text import CountVectorizer, HashingVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.preprocessing import LabelEncoder

In [14]:
le = LabelEncoder()
train_enc_labels = le.fit_transform(train_label)

In [15]:
le.classes_

array(['ADJ', 'ADP', 'ADV', 'AUX', 'CCONJ', 'DET', 'INTJ', 'NOUN',
       'NO_TAG', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM',
       'VERB', 'X'], dtype='<U6')

In [16]:
hvectorizer = HashingVectorizer(ngram_range=(1, 4), analyzer='char', n_features=100)

In [17]:
for number, element in enumerate(test_token):
    if element == None:
        test_token[number] = 'Договор'

In [18]:
X_train = hvectorizer.fit_transform(train_token)
X_test = hvectorizer.transform(test_token)

X_train.shape

(426182, 100)

In [19]:
import xgboost
from xgboost import XGBClassifier

xgb_model = XGBClassifier()

In [20]:
xgb_model.fit(X_train, train_enc_labels)

XGBClassifier(objective='multi:softprob')

In [21]:
pred = xgb_model.predict(X_test)

In [22]:
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay, classification_report
test_enc_labels = le.fit_transform(test_label)

print(classification_report(test_enc_labels, pred))

              precision    recall  f1-score   support

           0       0.73      0.65      0.69     15103
           1       0.98      0.96      0.97     13717
           2       0.88      0.48      0.62      7783
           3       0.80      0.97      0.88      1390
           4       0.88      1.00      0.93      5672
           5       0.84      0.61      0.71      4265
           6       0.70      0.29      0.41        24
           7       0.66      0.91      0.76     36238
           8       1.00      0.77      0.87       265
           9       0.81      0.64      0.72      1734
          10       0.95      0.74      0.83      5125
          11       0.78      0.86      0.82      7444
          12       0.83      0.14      0.24      5473
          13       0.98      1.00      0.99     29186
          14       0.81      0.90      0.85      2865
          15       1.00      0.85      0.92        62
          16       0.81      0.63      0.71     17110
          17       0.36    

Все реализованные методы показывают абсолютно сопоставимые результаты. Если нет ошибки в реализации самих методов, то лучше использовать самый "дешёвый" метод

####Задание 2. Проверить, насколько хорошо работает NER
Данные брать из http://www.labinform.ru/pub/named_entities/
- проверить NER из nltk/spacy/deeppavlov.
- написать свой NER, попробовать разные подходы.
  * передаём в сетку токен и его соседей.
  * передаём в сетку только токен.
  * свой вариант.
- сравнить свои реализованные подходы на качество — вывести precision/recall/f1_score.


In [23]:
!pip install corus
!pip install razdel

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [24]:
!wget http://www.labinform.ru/pub/named_entities/collection5.zip

--2022-09-02 18:08:37--  http://www.labinform.ru/pub/named_entities/collection5.zip
Resolving www.labinform.ru (www.labinform.ru)... 95.181.230.181
Connecting to www.labinform.ru (www.labinform.ru)|95.181.230.181|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1899530 (1.8M) [application/zip]
Saving to: ‘collection5.zip.2’

collection5.zip.2   100%[===================>]   1.81M  2.08MB/s    in 0.9s    

2022-09-02 18:08:38 (2.08 MB/s) - ‘collection5.zip.2’ saved [1899530/1899530]



In [25]:
!unzip collection5.zip

Archive:  collection5.zip
replace Collection5/001.ann? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
  inflating: Collection5/001.ann     
  inflating: Collection5/001.txt     
  inflating: Collection5/002.ann     
  inflating: Collection5/002.txt     
  inflating: Collection5/003.ann     
  inflating: Collection5/003.txt     
  inflating: Collection5/004.ann     
  inflating: Collection5/004.txt     
  inflating: Collection5/005.ann     
  inflating: Collection5/005.txt     
  inflating: Collection5/006.ann     
  inflating: Collection5/006.txt     
  inflating: Collection5/007.ann     
  inflating: Collection5/007.txt     
  inflating: Collection5/008.ann     
  inflating: Collection5/008.txt     
  inflating: Collection5/009.ann     
  inflating: Collection5/009.txt     
  inflating: Collection5/010.ann     
  inflating: Collection5/010.txt     
  inflating: Collection5/011.ann     
  inflating: Collection5/011.txt     
  inflating: Collection5/012.ann     
  inflating: Collection5/012.tx

In [26]:
from corus import load_ne5

path_col5 = 'Collection5/'
records = load_ne5(path_col5)

In [27]:
from razdel import tokenize

words_docs = []
for idx, rec in enumerate(records):
    words = []
    for token in tokenize(rec.text):
        type_ent = 'OUT'
        for ent in rec.spans:
            if (token.start >= ent.start) and (token.stop <= ent.stop):
                type_ent = ent.type
                break
        words.append([token.text, type_ent])
    words_docs.extend(words)

In [28]:
import pandas as pd

df_words = pd.DataFrame(words_docs, columns=['word', 'tag'])
df_words['tag'].value_counts()

OUT         219214
PER          21200
ORG          13651
LOC           4568
GEOPOLIT      4356
MEDIA         2482
Name: tag, dtype: int64

In [29]:
words_list = []
for word in words_docs:
    words_list.append(word[0])

words_list[:10]

['Вице-спикер',
 'Сейма',
 'Литвы',
 'уволился',
 'из-за',
 'превышения',
 'скорости',
 'Вице-спикер',
 'Сейма',
 'Литвы']

In [30]:
import nltk
nltk.download('averaged_perceptron_tagger')

nltk_pos_tags = nltk.pos_tag(words_list)

[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [31]:
nltk.download('maxent_ne_chunker')
nltk.download('words')

chunks = nltk.ne_chunk(nltk_pos_tags, binary=False)

[nltk_data] Downloading package maxent_ne_chunker to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package maxent_ne_chunker is already up-to-date!
[nltk_data] Downloading package words to /root/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [32]:
pos_tag = nltk.pos_tag(words_list)
chunk = nltk.ne_chunk(pos_tag, binary=False)  
entities = []
labels = []
for chunk in chunks:
    if hasattr(chunk, 'label'):
        for c in chunk:
            entities.append(c[0])
            labels.append(chunk.label())
    else:
        entities.append(chunk[0])
        labels.append('OUT')

In [33]:
entities_labels = list(zip(entities, labels))
entities_df = pd.DataFrame(entities_labels, columns = ['Entities', 'Labels'])

entities_df

,Entities,Labels
0,Вице-спикер,OUT
1,Сейма,PERSON
2,Литвы,PERSON
3,уволился,OUT
4,из-за,OUT
...,...,...
265466,работу,OUT
265467,П,OUT
265468,.,OUT
265469,Бородину,OUT


In [34]:
df_words['nltk_ner'] = entities_df['Labels']
df_words

,word,tag,nltk_ner
0,Вице-спикер,OUT,OUT
1,Сейма,ORG,PERSON
2,Литвы,GEOPOLIT,PERSON
3,уволился,OUT,OUT
4,из-за,OUT,OUT
...,...,...,...
265466,работу,OUT,OUT
265467,П,PER,OUT
265468,.,PER,OUT
265469,Бородину,PER,OUT


In [35]:
def ner_metrics(df, true_tag, ner_tag, column_name_1, column_name_2):
    right = 0
    fault = 0
    fault_2 = 0

    for index, person in  df.iterrows():
        if (person[column_name_1] == true_tag) & (person[column_name_2] == ner_tag):
            right += 1
        elif (person[column_name_1] == true_tag) & (person[column_name_2] !=  ner_tag):
            fault += 1
        elif (person[column_name_1] != true_tag) & (person[column_name_2] ==  ner_tag):
            fault_2 += 1
    precision = right / (right+fault_2)
    recall = right / (right+fault)
    f1 = (2*precision*recall) / (precision+recall)
    return print(f'precision = {precision}, recall = {recall}, f1_score = {f1}')

In [36]:
ner_metrics(df_words, 'PER', 'PERSON', 'tag', 'nltk_ner')

precision = 0.5961587708066581, recall = 0.43924528301886795, f1_score = 0.5058120586637697


In [37]:
ner_metrics(df_words, 'ORG', 'ORGANIZATION', 'tag', 'nltk_ner')

precision = 0.36092567868268804, recall = 0.05940956706468391, f1_score = 0.10202541200150962


In [38]:
!python -m spacy download ru_core_news_md

2022-09-02 18:18:01.863798: E tensorflow/stream_executor/cuda/cuda_driver.cc:271] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 41.9 MB 1.4 MB/s 
✔ Download and installation successful
You can now load the package via spacy.load('ru_core_news_md')


In [39]:
import spacy

nlp = spacy.load('ru_core_news_md')

labels=[]
i=0
for word in words_list:
    spacy_ner = nlp(word)
    if spacy_ner.__getitem__(0).ent_type_ == '':
        labels.append('OUT')
    else:
        labels.append(spacy_ner.__getitem__(0).ent_type_)
    i+=1
    if (i % 10000) == 0:
        print(f'обработано {i} слов')

обработано 10000 слов
обработано 20000 слов
обработано 30000 слов
обработано 40000 слов
обработано 50000 слов
обработано 60000 слов
обработано 70000 слов
обработано 80000 слов
обработано 90000 слов
обработано 100000 слов
обработано 110000 слов
обработано 120000 слов
обработано 130000 слов
обработано 140000 слов
обработано 150000 слов
обработано 160000 слов
обработано 170000 слов
обработано 180000 слов
обработано 190000 слов
обработано 200000 слов
обработано 210000 слов
обработано 220000 слов
обработано 230000 слов
обработано 240000 слов
обработано 250000 слов
обработано 260000 слов


In [40]:
df_words['spacy_ner'] = labels
df_words

,word,tag,nltk_ner,spacy_ner
0,Вице-спикер,OUT,OUT,OUT
1,Сейма,ORG,PERSON,OUT
2,Литвы,GEOPOLIT,PERSON,LOC
3,уволился,OUT,OUT,OUT
4,из-за,OUT,OUT,OUT
...,...,...,...,...
265466,работу,OUT,OUT,OUT
265467,П,PER,OUT,OUT
265468,.,PER,OUT,OUT
265469,Бородину,PER,OUT,PER


In [41]:
ner_metrics(df_words, 'PER', 'PER', 'tag', 'spacy_ner')

precision = 0.9747936849659131, recall = 0.5125943396226416, f1_score = 0.6718807963398046


In [42]:
ner_metrics(df_words, 'ORG', 'ORG', 'tag', 'spacy_ner')

precision = 0.6543102195086495, recall = 0.3297194344736649, f1_score = 0.4384802727715538


In [43]:
ner_metrics(df_words, 'LOC', 'LOC', 'tag', 'spacy_ner')

precision = 0.22163773452568825, recall = 0.276707530647986, f1_score = 0.246129880245351
